In [14]:
import keras
import numpy as np
import tensorflow as tf
from keras.src.optimizers import Adam
from keras.src.layers import Dense
from keras import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.src.layers import Dropout
from keras.src.layers import BatchNormalization

In [ ]:
# Load dataset
train_X = 
train_Y = 
val_X =
val_Y =
test_X =
test_Y =

In [7]:
# Define input shape and number of classes
INPUT_SHAPE = (200, 200, 3)  # Example input shape (image dimensions and channels)
NUM_CLASSES = 6  # Number of output classes (furniture types)

In [15]:
# Define hyperparameter grid
param_grid = {
    'num_layers': [3, 4, 5],
    'num_neurons': [64, 128, 256],
    'learning_rate': [0.001, 0.01],
    'epochs': [10, 20],
    'batch_size': [64],
    'dropout_rate': [0.2, 0.3, 0.4]
}

In [8]:
# Define base model
def create_model(num_layers=1, num_neurons=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(Dense(num_neurons, activation='relu', input_shape=INPUT_SHAPE))
    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons, activation='relu', kernel_initializer=keras.regularizers.l1(0.01)))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Create model estimator
model_estimator = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
# Perform randomized search using the estimator
grid_search = GridSearchCV(estimator=model_estimator, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=6)
grid_result = grid_search.fit(train_X, train_Y)

In [ ]:
print("Best Accuracy:", grid_result.best_score_)
print("Best Parameters:", grid_result.best_params_)

In [ ]:
# Get the best model
model = create_model(**grid_result.best_params_)
model.fit(train_X, train_Y, verbose=2)

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_X, test_Y, verbose=0)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Evaluate the model on the validation dataset
val_loss, val_acc = model.evaluate(val_X, val_Y, verbose=0)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")